# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
# load csv
city_data = "Weather_data.csv"
city_df = pd.read_csv(city_data)
city_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Jinchang,38.50,102.17,60.91,45,100,27.36,CN,1598652218
1,1,Noumea,-22.28,166.46,68.00,82,100,1.12,NC,1598652185
2,2,Rikitea,-23.12,-134.97,68.83,67,0,5.48,PF,1598652063
3,3,Arraial do Cabo,-22.97,-42.03,71.60,83,0,14.99,BR,1598652091
4,4,Japura,-0.32,102.35,73.54,89,100,2.21,ID,1598652219
...,...,...,...,...,...,...,...,...,...,...
550,550,‘Ewa Beach,21.32,-158.01,87.80,55,75,9.17,US,1598652361
551,551,Guilin,25.28,110.29,73.40,94,0,2.24,CN,1598652361
552,552,Grindavik,63.84,-22.43,48.20,100,90,13.87,IS,1598652362
553,553,Solnechnyy,50.72,136.63,52.05,71,6,6.51,RU,1598652362


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
# configu gmaps
gmaps.configure(api_key=g_key)
location = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"]

In [59]:
# Plot Heatmap
fig = gmaps.figure(center = (46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
narrow_cities = city_df.loc[(city_df["Wind Speed"] <=10) & (city_df["Cloudiness"] == 0) & \
                                 (city_df["Temperature"] >= 70) & (city_df["Temperature"] <= 80)]

narrow_cities = narrow_cities.dropna()
narrow_cities

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,Bereket,39.24,55.52,74.93,21,0,8.12,TM,1598652223
71,71,San Fernando del Valle de Catamarca,-28.47,-65.79,78.01,31,0,1.01,AR,1598652042
119,119,Ardakān,32.31,54.02,78.80,15,0,4.70,IR,1598652247
122,122,Marrakesh,31.63,-8.01,78.80,50,0,9.17,MA,1598652049
174,174,Caravelas,-17.71,-39.25,70.79,79,0,4.21,BR,1598652260
184,184,Nanyang,32.99,112.53,70.36,88,0,4.65,CN,1598652262
202,202,Barkhan,29.90,69.53,77.43,65,0,2.98,PK,1598652267
330,330,Cassilândia,-19.11,-51.73,79.34,23,0,5.50,BR,1598652302
350,350,Tūkrah,32.53,20.58,76.95,76,0,7.16,LY,1598652308
414,414,Tshikapa,-6.42,20.80,77.86,60,0,0.76,CD,1598652324


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
# cleaned list
hotel_df = narrow_cities.loc[:,["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Hotel Name
20,Bereket,TM,39.24,55.52,
71,San Fernando del Valle de Catamarca,AR,-28.47,-65.79,
119,Ardakān,IR,32.31,54.02,
122,Marrakesh,MA,31.63,-8.01,
174,Caravelas,BR,-17.71,-39.25,


In [55]:
for index, row in hotel_df.iterrows():
    print(row)
    target_coordinates = str(row['Latitude']) + ',' + str(row['Longitude'])
    target_search = "Hotel"
    target_radius = 5000,
    target_type = "hotel"
# set parameters  
    params = {
        "radius": 5000,
        "type": target_type,
        "keyword": target_search,
        "location": target_coordinates,
        "key" : g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    response = requests.get(base_url, params = params)
    
    

City          Bereket
Country            TM
Latitude        39.24
Longitude       55.52
Hotel Name           
Name: 20, dtype: object
City          San Fernando del Valle de Catamarca
Country                                        AR
Latitude                                   -28.47
Longitude                                  -65.79
Hotel Name                                       
Name: 71, dtype: object
City          Ardakān
Country            IR
Latitude        32.31
Longitude       54.02
Hotel Name           
Name: 119, dtype: object
City          Marrakesh
Country              MA
Latitude          31.63
Longitude         -8.01
Hotel Name             
Name: 122, dtype: object
City          Caravelas
Country              BR
Latitude         -17.71
Longitude        -39.25
Hotel Name             
Name: 174, dtype: object
City          Nanyang
Country            CN
Latitude        32.99
Longitude      112.53
Hotel Name           
Name: 184, dtype: object
City          Barkhan
Country   

In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))